<a href="https://colab.research.google.com/github/midronij/CSC326Lab/blob/master/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

def loadData():
    with np.load('notMNIST.npz') as dataset:
        Data, Target = dataset['images'], dataset['labels']
        posClass = 2
        negClass = 9
        dataIndx = (Target==posClass) + (Target==negClass)
        Data = Data[dataIndx]/255.
        Target = Target[dataIndx].reshape(-1, 1)
        Target[Target==posClass] = 1
        Target[Target==negClass] = 0
        np.random.seed(421)
        randIndx = np.arange(len(Data))
        np.random.shuffle(randIndx)
        Data, Target = Data[randIndx], Target[randIndx]
        trainData, trainTarget = Data[:3500], Target[:3500]
        validData, validTarget = Data[3500:3600], Target[3500:3600]
        testData, testTarget = Data[3600:], Target[3600:]
    return trainData, validData, testData, trainTarget, validTarget, testTarget

def loss(w, b, x, y, reg):
  z = np.matmul(np.transpose(w), x) + b
  y_est = 1 / (1 + np.exp(-1 * z))

  e = -1 * y * np.log(y_est) - (1 - y) * np.log(1 - y_est)
  E = 1 / np.ndarray.size(x) * np.sum(e)
  answer += (reg / 2) * np.linalg.norm(w)^2 

  return answer

def grad_loss(w, b, x, y, reg):
  z = np.matmul(np.transpose(w), x) + b

  #gradient with respect to w
  grad_e_w = x * y * np.exp(z) / (1 + np.exp(z)) - x * y + (1 - y) * x * np.exp(z) / (1 + np.exp(z))
  grad_E_w = 1 / np.ndarray.size(x) * np.sum(grad_e_w) + reg * w

  #gradient with respect to b
  grad_e_b = y * np.exp(z) / (1 + np.exp(z)) - y + (1 - y) * np.exp(z) / (1 + np.exp(z))
  grad_E_b = 1 / np.ndarray.size(x) * np.sum(grad_e_b) + reg * w

  return grad_E_w, grad_E_b

def grad_descent(W, b, x_training, y_training, x_validation, y_validation, x_test, y_test, alpha, epochs, reg, error_tol):
  Wold = W
  bold = b
  for i in range(epochs):
    #losses
    loss_training = loss(Wold, bold, x_training, y_training, reg) #training loss
    loss_validation = loss(Wold, bold, x_validation, y_validation, reg) #validation loss
    loss_test = loss(Wold, bold, x_test, y_test, reg)#test loss

    #accuracy
    #training accuracy
    z_training = np.matmul(np.transpose(Wold), x_training) + bold
    y_est_training = 1 / (1 + np.exp(-1 * z_training))
    accuracy_training = ((y_est_training >= 0.5) == y_training) / ndarray.size(y_training) #check number of correct predictions

    #validation accuracy
    z_validation = np.matmul(np.transpose(Wold), x_validation) + bold
    y_est_validation = 1 / (1 + np.exp(-1 * z_validation))
    accuracy_validation = ((y_est_validation >= 0.5) == y_validation) / ndarray.size(y_validation) #check number of correct predictions

    #test accuracy
    z_test = np.matmul(np.transpose(Wold), x_test) + bold
    y_est_test = 1 / (1 + np.exp(-1 * z_test))
    accuracy_test = ((y_est_test >= 0.5) == y_test) / ndarray.size(y_test) #check number of correct predictions

    grad_w, grad_b = grad_loss(Wold, bold, x_training, y_training, reg)
    Wnew = Wold - alpha * grad_w
    bnew = bold - alpha * grad_b

    if np.linalg.norm(Wnew - Wold) < error_tol:
      return Wnew, bnew
    
    Wold = Wnew
    bold = bnew

  return Wnew, bnew


